In [1]:
import pandas as pd
import numpy as np
import utils
import Dotblot
import GeneralDilution

from importlib import reload
import os
import re
import json

In [ ]:
reload(utils)
reload(Dotblot)
dotblot_method = Dotblot.DotblotMethod()

data = utils.import_excel_dotblot(r"L:/Departements/BTDS_AD/002_AFFS/Lab Automation/09. Tecan/06. DotBlot_automation_DPP/DotBlot_automation_dilution_template_final.xlsx")
sample_dilution_data = data[0]
coating_protein_dilution_data = data[1]
pos_control_dilution_data = data[2]
neg_control_dilution_data = data[3]

In [13]:
dotblot_method.pos_control_buffer = "Assay"
# dotblot_method.pos_control_vial_type = 1
dotblot_method.n_pos_control_steps = len(pos_control_dilution_data["Assay buffer volume"])
dotblot_method.sample_dilution_data = {col: sample_dilution_data[col].values.astype(float).tolist() for col in sample_dilution_data.columns}
dotblot_method.pos_control_dilution_data = {col: pos_control_dilution_data[col].values.astype(float).tolist() for col in pos_control_dilution_data.columns}
dotblot_method.pos_control_vial_posX = 1
dotblot_method.main_sample_labware_type = "Falcon15"
dotblot_method.main_sample_dil_destination = "DeepWell"
dotblot_method.n_samples_main_dilution = 7
dotblot_method.n_sample_dilution_steps = len(sample_dilution_data["Assay buffer volume"])

dotblot_method.dotblot()

pos dilutions done
neg dilutions done
labdest: ['96 Deep Well 2ml[001]' '96 Deep Well 2ml[001]' '96 Deep Well 2ml[001]'
 '96 Deep Well 2ml[001]' '96 Deep Well 2ml[001]' '96 Deep Well 2ml[001]'
 '96 Deep Well 2ml[001]']
destwell: [3 4 5 6 7 8 9]
before long csv
sample dilutions done
sample eppendorf pos: [5, 6, 11, 12, 15, 16, 17, 18, 21, 22, 27, 28, 31, 32]
calculated pump labware positions
generated all GWL files


([1], [5, 6, 11, 12, 15, 16, 17, 18, 21, 22, 27, 28, 31, 32])

In [20]:
csv_path = r'L:\Departements\BTDS_AD\002_AFFS\Lab Automation\09. Tecan\01. Methods\3. DotBlot\2. Positive control.csv'

mix_data = []
labware_name = "Custom holder[001]"
labware_pos = 1
volume = 300

for i in range(3):
    mix_data.append([labware_name, labware_pos, labware_name, labware_pos, volume])

mix_data
mix_csv = pd.DataFrame(mix_data, columns=None)
mix_csv

original_csv = pd.read_csv(csv_path, header=None) # create dataframe from original CSV file
final_csv = pd.concat([original_csv, mix_csv], ignore_index=True) # concatenate both dataframes into one
final_csv.to_csv(csv_path, index=False, header=False)

In [ ]:
pump_step_data_simple = [{'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Vacuum'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}]
pump_step_data = [{'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples', 'wells_type': 'All wells'}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Coating protein', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'Blocking buffer', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Pos/Neg control', 'wells_type': 'Only samples'}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Conjugate', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Vacuum'}]

pump_step_data

In [14]:
timer_step_data = pd.DataFrame([], columns=["wait_timer"])

for i, step in enumerate(pump_step_data):
    # print(step)
    if step["step_type"] == "Transfer volume to wells":
        pass
        # step_types.append("transfer")
        # transfer_step_data.loc[i] = (list(step.values())[1:]) # select only values from dict, ignoring first entry (step type)

        # self.transfer_volume_to_wells(step["liquid_type"], self.pump_lw_name, step["volume_amount"], csv_number)
        # csv_number = csv_number + 1

    elif step["step_type"] == "Vacuum":
        pass
        # step_types.append("vacuum")
        # vacuum_step_data.loc[i] = (list(step.values())[1:]) # select only values from dict, ignoring first entry (step type)
    elif step["step_type"] == "Wait timer":
        print(step)
        pass
        # step_types.append("timer")
        # multiply by 60 because in Tecan the timer is in seconds and here the value is in minutes
        timer_step_data.loc[i] = (list(step.values())[1:]) # select only values from dict, ignoring first entry (step type)

timer_step_data["wait_timer"] = timer_step_data["wait_timer"] * 60

timer_step_data


{'step_type': 'Wait timer', 'wait_timer': 10.0}
{'step_type': 'Wait timer', 'wait_timer': 10.0}
{'step_type': 'Wait timer', 'wait_timer': 10.0}
{'step_type': 'Wait timer', 'wait_timer': 5.0}
{'step_type': 'Wait timer', 'wait_timer': 5.0}
{'step_type': 'Wait timer', 'wait_timer': 5.0}


,wait_timer
3,600.0
6,600.0
11,600.0
14,300.0
16,300.0
18,300.0


### For Dotblot first try for TV-TMD-35372

In [5]:
steps = [
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Coating protein'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'Blocking buffer'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Pos/Neg control'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples'}, 
         {'step_type': 'Wait timer', 'wait_timer': 600}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Conjugate'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'},  
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'},  
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'}
]

steps

[{'step_type': 'Transfer volume to wells',
  'volume_amount': '200',
  'liquid_type': 'DPBS'},
 {'step_type': 'Wait timer', 'wait_timer': 60},
 {'step_type': 'Vacuum'},
 {'step_type': 'Transfer volume to wells',
  'volume_amount': '100',
  'liquid_type': 'Coating protein'},
 {'step_type': 'Wait timer', 'wait_timer': 480},
 {'step_type': 'Vacuum'},
 {'step_type': 'Transfer volume to wells',
  'volume_amount': '200',
  'liquid_type': 'Blocking buffer'},
 {'step_type': 'Wait timer', 'wait_timer': 480},
 {'step_type': 'Vacuum'},
 {'step_type': 'Transfer volume to wells',
  'volume_amount': '100',
  'liquid_type': 'Pos/Neg control'},
 {'step_type': 'Transfer volume to wells',
  'volume_amount': '100',
  'liquid_type': 'Samples'},
 {'step_type': 'Wait timer', 'wait_timer': 600},
 {'step_type': 'Vacuum'},
 {'step_type': 'Transfer volume to wells',
  'volume_amount': '100',
  'liquid_type': 'Conjugate'},
 {'step_type': 'Wait timer', 'wait_timer': 480},
 {'step_type': 'Vacuum'},
 {'step_type': 

In [19]:
import json

with open('assays.json') as f:
    d = json.load(f)

print(d)

{'assays': [{'TV-TMD-35372': {'assay_type': 'dotblot', 'title': 'Test Method for Determining the Identity of JNJ80948543 using the Dot Blot Assay', 'step_types': [{'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, {'step_type': 'Wait timer', 'wait_timer': 60}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Coating protein'}, {'step_type': 'Wait timer', 'wait_timer': 480}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'Blocking buffer'}, {'step_type': 'Wait timer', 'wait_timer': 480}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Pos/Neg control'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples'}, {'step_type': 'Wait timer', 'wait_timer': 600}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '

In [28]:
assay_names = []
for assay in d["assays"]:
    assay_names.append(list(assay.keys())[0])

assay_names

['TV-TMD-35372', 'test']

In [ ]:
dotblot_method.pump_steps_data = pump_step_data
dotblot_method.sample_eppendorf_positions = [3,4,5,6]

dotblot_method.generate_pump_step_instruction_files()

In [4]:
dotblot_method.transfer_volume_to_wells(dotblot_method.dpbs_lw_name, "96 Deep Well[001]", 100)

In [68]:
pos = 9
# each block has 24 wells, and therefore fits 8 samples in vertical order
block = int((pos-1) / 8) # floor operation
row = pos % 8
if row == 0:
    row = 8

init_pos = block * 24 + row

pos = [init_pos, init_pos+8, init_pos+16]

pos

[25, 33, 41]

In [73]:
dotblot_method.pump_lw_well_pos["samples_pos"]

[[3, 11, 19], [4, 12, 20], [5, 13, 21], [6, 14, 22]]

In [6]:
step_types = [{'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'DPBS'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples'}, {'step_type': 'Wait timer', 'wait_timer': 6.0}, {'step_type': 'Vacuum'}]
step_types

[{'step_type': 'Transfer volume to wells',
  'volume_amount': '100',
  'liquid_type': 'DPBS'},
 {'step_type': 'Wait timer', 'wait_timer': 10.0},
 {'step_type': 'Vacuum'},
 {'step_type': 'Transfer volume to wells',
  'volume_amount': '100',
  'liquid_type': 'Samples'},
 {'step_type': 'Wait timer', 'wait_timer': 6.0},
 {'step_type': 'Vacuum'}]

#### GENERAL DILUTION TESTS

In [182]:
reload(utils)
reload(GeneralDilution)

gen_dil = GeneralDilution.GeneralDilution()

In [183]:
sample_dilution_data = {'Dilution step': [1.0, 2.0, 3.0], 'Initial concentration': [150.0, 0.7, 0.003], 'Withdrawn volume': [4.0, 4.0, 30.0], 'Final concentration': [0.7, 0.003, 0.0001], 'Assay buffer volume': [853.1428571428572, 929.3333333333333, 869.9999999999999], 'Sample volume': [4.0, 4.0, 30.0], 'Final Volume': [857.1428571428572, 933.3333333333333, 899.9999999999999]}
gen_dil.sample_dilution_data = sample_dilution_data
gen_dil.n_sample_dilution_steps = len(gen_dil.sample_dilution_data["Assay buffer volume"])
gen_dil.n_samples = 7
gen_dil.buffer_type = "DPBS"
gen_dil.sample_lw_origin = "Falcon15"
gen_dil.sample_lw_dest = "2R Vial"
gen_dil.used_labware_pos = dict.fromkeys(gen_dil.last_labware_pos, 0)

In [184]:
gen_dil.general_dilution()

[1, 2, 3, 4, 5, 6, 7]

In [185]:
gen_dil.used_labware_pos

{'Falcon15': 7,
 'Falcon50': 0,
 'Eppendorf': 0,
 'DeepWell': 21,
 '2R Vial': 7,
 '8R Vial': 0,
 'CustomVialHolder': 0,
 'AssayBuffer': 0,
 'DPBS': 0}

In [125]:
gen_dil.next_labware_pos(utils.LabwareNames["Falcon50"])

2

### VOLUME TRANSFER

In [51]:
import VolumeTransfer
from importlib import reload
from utils import *

In [49]:
reload(VolumeTransfer)
vol_tr = VolumeTransfer.VolumeTransfer()

In [ ]:
VolumeTransfer.dilution_position_def()

In [52]:
vol_tr.sample_lw_origin = "Eppendorf"
vol_tr.n_samples = 3
vol_tr.sample_lw_dest = "DeepWell"
vol_tr.n_sample_repetitions = 4
vol_tr.volume_transfered = 50
vol_tr.used_labware_pos = dict.fromkeys(vol_tr.used_labware_pos, 0) # reset dict

vol_tr.volume_transfer()

NameError: name 'dilution_position_def' is not defined

#### TESTS


In [34]:
step_types = []

step_types.append("timer")


step_types.append("end")

pd.DataFrame(step_types).to_csv("config - step_types.csv", header=["step_type"], index=False, lineterminator=";\n") # generate CSV file


In [57]:
val = {lw: 1 for lw in utils.LabwareNames}

val


{'Falcon15': 1,
 'Falcon50': 1,
 'Eppendorf': 1,
 'DeepWell': 1,
 '_2R_Holder': 1,
 '_8R_Holder': 1,
 'CustomVialHolder': 1,
 'AssayBuffer': 1,
 'DPBS': 1}

In [1]:
# examen ivan ej 1

lista = [3, 2, 6, 6, 5, 3, 2, 8, 9]

if len(lista) <= 20:
    res = []
    prev = 0
    sum = 0
    for num in lista:
        if num >= prev:
            sum = sum + num

        else:
            if sum > 0:
                res.append(sum)
            else:
                res.append(num)
                
        prev = num

print(lista)
print(res)


[3, 2, 6, 6, 5, 3, 2, 8, 9]
[2, 5, 3, 2]


## CSV to GWL converter

In [57]:
def convert_csv_to_gwl(input_file_path, output_file_path, onetime_tip_change=False):
    NUMBER_OF_PARTS = 5 # number of parts per instruction line in the csv file

    # Open the input file in read mode and output file in write mode
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
        # Read all lines from the input file
        lines = input_file.readlines()

        output_lines = []
        
        # Process each line in the input file
        for line in lines:
            # Strip any leading/trailing whitespace
            line = line.strip()
            if not line:
                continue  # Skip empty lines
            
            # Split the line by commas
            parts = line.split(',')
            if len(parts) != NUMBER_OF_PARTS:
                continue  # Skip lines that don't have exactly 6 parts
            
            # Extract the values
            first_part = parts[0]
            second_part = parts[1]
            third_part = parts[2]
            fourth_part = parts[3]
            fifth_part = parts[4]
            
            # Create the three lines for the output
            a_line = f"A;{first_part};;;{second_part};;{fifth_part};;;;\n"
            d_line = f"D;{third_part};;;{fourth_part};;{fifth_part};;;;\n"
            w_line = "W;\n" # use F to flush tip remaining contents instead of changing tip, which is actually better for each round
            
            # Append the lines to the output lines list
            output_lines.append(a_line)
            output_lines.append(d_line)
            output_lines.append(w_line)
        
        # If True, the W/F commands will be removed so that only 8 are left, to ensure that all tips are used and all are reused throughout the script,
        # eliminating the unneeded W/F commands and leaving then as evenly distributed as possible.
        if onetime_tip_change:
            # Count the number of "W;" lines
            w_lines_count = sum(1 for line in output_lines if line == "W;\n")
            
            if w_lines_count > 8:
                # Calculate the indices where the "W;" lines should be kept
                total_lines = len(output_lines)
                indices_to_keep = [int(i * (w_lines_count - 1) / 7) for i in range(8)]
                
                # Collect the indices of all "W;" lines
                w_indices = [i for i, line in enumerate(output_lines) if line == "W;\n"]
                
                # Determine which "W;" lines to keep based on calculated indices
                keep_w_indices = [w_indices[i] for i in indices_to_keep]
                
                # Remove "W;" lines that are not in the keep_w_indices
                new_output_lines = [line for i, line in enumerate(output_lines) if line != "W;\n" or i in keep_w_indices]
            else:
                new_output_lines = output_lines
        
        else:
            new_output_lines = output_lines

        # Replace all "W;" with "F;" except the last "W;"
        w_indices = [i for i, line in enumerate(new_output_lines) if line == "W;\n"]
        for i in w_indices[:-1]:  # Exclude the last "W;"
            new_output_lines[i] = "F;\n"

        # Write the lines to the output file
        output_file.writelines(new_output_lines)


In [26]:
# Usage
input_file_path = '1input.csv'  # Replace with the path to your input file
output_file_path = '1output.gwl'  # Replace with the path to your output file

convert_csv_to_gwl(input_file_path, output_file_path)

In [58]:
def convert_all_files_in_directory(path, pattern):
    # Define the input file pattern
    # file_pattern = re.compile(r'3\. Pump steps - Transfer (\d+)\.csv')
    file_pattern = re.compile(pattern)
    
    # Iterate over all files in the directory
    for filename in os.listdir(path):

        # Check if the filename matches the pattern
        match = file_pattern.match(filename)
        if match:
            # Generate the input and output file paths
            input_file_path = os.path.join(path, filename)
            output_file_path = os.path.join(path, f"{filename.replace('.csv', '.gwl')}")
            
            # Convert the file
            convert_csv_to_gwl(input_file_path, output_file_path)

In [59]:
# Usage
directory_path = r'L:\Departements\BTDS_AD\002_AFFS\Lab Automation\09. Tecan\01. Methods\3. DotBlot'  # Replace with the path to your directory
convert_all_files_in_directory(directory_path, r"3\. Pump steps - Transfer (\d+)\.csv")

### JSON tests

In [23]:
# read json file
with open('assays.json', "r") as f:
    RAW_ASSAYS_DATA = json.load(f)["assays"]

In [24]:
# METHODS_LIST = ["Dotblot", "FIPA", "DLS", "nDSF"]
# PRODUCTS_DICT = {
#     "Dotblot": ["PD1", "Product1_B"],
#     "FIPA": ["Product2_A", "Product2_B"],
#     "DLS": ["Product3_A", "Product3_B"],
#     "nDSF": ["Product3_A", "Product3_B"],
# }
METHODS_LIST = []
PRODUCTS_DICT = {}

for assay in RAW_ASSAYS_DATA:
    if assay["method"] not in METHODS_LIST:
        METHODS_LIST.append(assay["method"])
    
    if assay["method"] not in PRODUCTS_DICT:
        PRODUCTS_DICT[assay["method"]] = [] # create empty list
        
    PRODUCTS_DICT[assay["method"]].append(assay["product"]) # add product to list

print("methods:", METHODS_LIST)
print("products", PRODUCTS_DICT)


methods: ['Dotblot', 'FIPA']
products {'Dotblot': ['pd1', 'pd1', 'CD79'], 'FIPA': ['product3']}


In [19]:
def get_indexes(raw_data, method, product):
    # Find indexes of matching assays
    matching_indexes = [
        index for index, assay in enumerate(raw_data)
        if assay.get('method') == method and assay.get('product') == product
    ]

    return matching_indexes

In [32]:
method = "Dotblot"
product = "CD79"

indices = get_indexes(RAW_ASSAYS_DATA, method, product)
indices

[3]

In [34]:
RAW_ASSAYS_DATA[indices[0]]["tmd"]

'test tmd'

## Calculate labware type for each reagent based on volumes used in the method

In [38]:
step_types = [
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60},
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Coating protein"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "Blocking buffer"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Pos/Neg control"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Samples"}, 
                {"step_type": "Wait timer", "wait_timer": 600}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Conjugate"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"},  
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"},  
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"}]

REAGENTS = ["samples", "pos_ctr", "neg_ctr", "dpbs", "conjugate", "blocking_buffer", "coating_protein"]
n_samples = 4

In [52]:
total_volume = {}

for step in step_types:
    if step["step_type"] == "Transfer volume to wells":
        if step["liquid_type"] not in total_volume:
            total_volume[step["liquid_type"]] = 0 # create key
        
        total_volume[step["liquid_type"]] = total_volume[step["liquid_type"]] + float(step["volume_amount"])/1000 # because volume amount is in uL and we want it in mL

total_volume

{'DPBS': 0.8,
 'Coating protein': 0.1,
 'Blocking buffer': 0.2,
 'Pos/Neg control': 0.1,
 'Samples': 0.1,
 'Conjugate': 0.1}

In [53]:
for key in total_volume:
    total_volume[key] *= n_samples
    total_volume[key] *= 3 # because each sample is triplicated (placed in 3 wells)
    total_volume[key] = round(total_volume[key], 2)

total_volume

{'DPBS': 9.6,
 'Coating protein': 1.2,
 'Blocking buffer': 2.4,
 'Pos/Neg control': 1.2,
 'Samples': 1.2,
 'Conjugate': 1.2}

In [61]:
# labware name: dead_volume, max_volume (in mL)
LABWARE_INFO = {
    "Eppendorf": [0.2, 1.5],
    "Falcon15": [0.8, 15],
    "100mL_reservoir": [2, 100]
}

In [83]:
def find_best_container(volumes_dict, containers_dict):
    result = {}
    
    for item, volume_needed in volumes_dict.items():
        best_container = None
        best_sum = float('inf')
        
        for container, (vol, cap) in containers_dict.items():
            if volume_needed <= cap - vol:
                current_sum = vol + cap
                if current_sum < best_sum:
                    best_sum = current_sum
                    best_container = container
        
        if best_container:
            result[item] = best_container
        else:
            result[item] = "No suitable container"
    
    return result

In [87]:
def find_best_container(reagents_vol_dict: dict, labware_dict: dict):
    """
    Assigns the best labware type for each reagent considering the volume needed.
    """

    result = {}
    
    for item, volume_needed in reagents_vol_dict.items():
        best_container = None
        best_sum = float('inf')
        
        for labware, (dead_vol, max_vol) in labware_dict.items():
            if volume_needed <= max_vol - dead_vol:
                current_sum = max_vol - dead_vol
                if current_sum < best_sum:
                    best_sum = current_sum
                    best_container = labware
        
        if best_container:
            result[item] = best_container
        else:
            result[item] = "VOLUME TOO BIG"
    
    return result

In [97]:
total_volume["Conjugate"] = 14201

find_best_container(total_volume, LABWARE_INFO)

{'DPBS': 'Falcon15',
 'Coating protein': 'Eppendorf',
 'Blocking buffer': 'Falcon15',
 'Pos/Neg control': 'Eppendorf',
 'Samples': 'Eppendorf',
 'Conjugate': 'VOLUME TOO BIG'}